In [ ]:
#IMPORT NECESSARY PACKAGES
import CO2_functions
import CO2_Processing
import pandas as pd
from CO2_functions import *
from CO2_Processing import *
import matplotlib.pyplot as plt
import pickle
import numpy as np
pd.options.display.max_rows = 999
from datetime import datetime
import os

In [ ]:
import importlib
importlib.reload(CO2_functions)
importlib.reload(CO2_Processing)

from CO2_functions import *
from CO2_Processing import *

In [ ]:
data = retrieve_data_from_folder('../CO2_Data_Final','2019-11-06','2019-11-27')

In [ ]:
data = remove_spikes(pd.read_pickle('Spike_ETs.pkl'),data)
data = downsample_and_concatenate(data) 
data = combine_vent_data(data,1)
data['Vent_Mass'] = moving_mass_flow(data['Vent_Mass'])

In [ ]:
for i in range(1,4):
    data['Multi'].loc[(data['Multi']['CO2_{}'.format(i)] < 395)|(data['Multi']['CO2_{}'.format(i)] > 600)] = float('NaN')
data['Multi'].interpolate(limit=1,inplace=True)

In [ ]:
multi = data['Multi'].copy()
multi = multi.loc[multi['Multi_Loc']<2.0]

In [ ]:
#data['Multi'] = data['Multi'].resample('1T').agg(dict(Rotations='sum',Wind_Velocity='mean',Wind_Direction = 'mean'))


In [ ]:
plot_refinement_all([[data['Multi'],'Corrected_DT','CO2_3']],'separate')

In [ ]:
#multi = data['Multi']
multi = multi[['CO2_1','CO2_2','CO2_3','Temp','Rotations','Wind_Velocity','Wind_Direction','Multi_Loc','DOW']]
multi = multi_direction_correction(multi)
multi.columns = ['CO2_1','CO2_2','CO2_3','Temp','Rotations','ws','wd','Multi_Loc','DOW']
multi1 = multi.loc[multi['ws']>2.0].dropna()

In [ ]:
#Import everything to view graphics
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.lib import grdevices
base = importr('base')
utils = importr('utils')
import IPython
from IPython.display import Image, display
graphics = importr('graphics')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
if not ro.packages.isinstalled('openair'):
    utils.install_packages('openair')

df = multi1

r=ro.r #set r object to "r"
r.library('openair') #load library openair

pandas2ri.activate() #activate the pandas to r dataframe function
r_dataframe = pandas2ri.py2ri(df) #convert pandas df to r df

with rpy2.robjects.lib.grdevices.render_to_bytesio(grdevices.png, width=900, height=700, res=150) as img: #graphical settings
    #r.polarAnnulus(r_dataframe,poll = pollutant,period='hour',main='Hour',width = 'fat')  # setup plot through r
    r.polarPlot(r_dataframe,poll='CO2_2',statistic='cpf',percentile=80)  # setup plot through r

IPython.display.display(IPython.display.Image(data=img.getvalue(), format='png', embed=True)) #display plot